# Data Acquisition and Preprocessing Term Project

##                                - Spotify Top 50 playlists Dataset

### 1. Spotipy Library

Spotipy is a lightweight Python library for the Spotify Web API. With Spotipy we can get full access to many music data provided by the Spotify platform.

In [0]:
# install spotipy library first
pip install spotipy

  Created wheel for spotipy: filename=spotipy-2.4.4-cp36-none-any.whl size=12336 sha256=25a288ba70c02d3ef320716c971e12d0507f964a10d028acfc96882714356067
  Stored in directory: /root/.cache/pip/wheels/76/28/19/a86ca9bb0e32dbd4a4f580870250f5aeef852870578e0427e6
Successfully built spotipy


In [0]:
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials

Get authorization from Spotify

In [0]:
username = '12977977'
scope = 'user-read-private'

client_id = '5fd7312f2d4348a196322a692ae3dd30'
client_secret = '8868bbd27bc845fd8bbce969996941af'

In [0]:
client_credentials_manager = SpotifyClientCredentials(client_id = client_id, client_secret = client_secret)
spotify = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

### 2. Get Top 50 playlists from 62 Countries - Base Dataset

#### 2.1 Get official country playlists ID

In [0]:
!pip install bs4

Use Beautiful Soup to do our web scraping

In [0]:
from bs4 import BeautifulSoup
import requests
import re
from pprint import pprint

We get the available area from "https://support.spotify.com/us/using_spotify/the_basics/full-list-of-territories-where-spotify-is-available/", where spotify users can sign up.

In [0]:
url = "https://support.spotify.com/us/using_spotify/the_basics/full-list-of-territories-where-spotify-is-available/"
html = requests.get(url).text

soup = BeautifulSoup(html, 'html.parser')
# Show what we ger from the web 
pprint(soup)

<!DOCTYPE html>

<!--[if lte IE 9]>    <html class="no-js ie9 " lang="en-US" dir="ltr"> <![endif]-->
<!--[if gt IE 9]><!--> <html class="no-js " dir="ltr" lang="en-US"> <!--<![endif]-->
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/><script type="text/javascript">(window.NREUM||(NREUM={})).loader_config={xpid:"VQQOWFVbDRAGVFBVAAADUQ==",licenseKey:"24c06e9b53",applicationID:"60177147"};window.NREUM||(NREUM={}),__nr_require=function(t,n,e){function r(e){if(!n[e]){var o=n[e]={exports:{}};t[e][0].call(o.exports,function(n){var o=t[e][1][n];return r(o||n)},o,o.exports)}return n[e].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<e.length;o++)r(e[o]);return r}({1:[function(t,n,e){function r(t){try{s.console&&console.log(t)}catch(n){}}var o,i=t("ee"),a=t(20),s={};try{o=localStorage.getItem("__nr_flags").split(","),console&&"function"==typeof console.log&&(s.console=!0,o.indexOf("dev")!==-1&&(s.dev=!0),o.indexOf("nr_dev")!==-1&&

In [0]:
def get_available_countries():
    url = "https://support.spotify.com/us/using_spotify/the_basics/full-list-of-territories-where-spotify-is-available/"
    html = requests.get(url).text

    soup = BeautifulSoup(html, 'html.parser')

    # available_countries[continent] = [country, country, country]
    available_countries = {}
    
    # The <td> tag defines a standard cell in an HTML table.
    # For example, 
    # <td colspan="1" rowspan="1">Africa</td><td colspan="1" rowspan="1">Algeria, Egypt, Morocco, South Africa, Tunisia.</td></tr><tr>
    target_tags = soup.find_all(['td'])
    
    for tag in target_tags:
        # match '[\w\s]+, only if it's followed by '[.,]'
        countries = re.findall('[\w\s]+(?=[.,])', tag.text)
        if countries:
            countries = [country.lstrip() for country in countries]
            available_countries[continent] = countries
        else:
            continent = tag.text

    return available_countries

In [0]:
# Available area list
get_available_countries()

{'Africa': ['Algeria', 'Egypt', 'Morocco', 'South Africa', 'Tunisia'],
 'Asia': ['Bahrain',
  'Hong Kong',
  'India',
  'Indonesia',
  'Israel',
  'Japan',
  'Jordan',
  'Kuwait',
  'Lebanon',
  'Malaysia',
  'Oman',
  'Palestine',
  'Philippines',
  'Qatar',
  'Saudi Arabia',
  'Singapore',
  'Taiwan',
  'Thailand',
  'United Arab Emirates',
  'Vietnam'],
 'Europe': ['Andorra',
  'Austria',
  'Belgium',
  'Bulgaria',
  'Cyprus',
  'Czech Republic',
  'Denmark',
  'Estonia',
  'Finland',
  'France',
  'Germany',
  'Greece',
  'Hungary',
  'Iceland',
  'Ireland',
  'Italy',
  'Latvia',
  'Liechtenstein',
  'Lithuania',
  'Luxembourg',
  'Malta',
  'Monaco',
  'Netherlands',
  'Norway',
  'Poland',
  'Portugal',
  'Romania',
  'Slovakia',
  'Spain',
  'Sweden',
  'Switzerland',
  'Turkey',
  'United Kingdom'],
 'North America': ['Canada',
  'Costa Rica',
  'Dominican Republic',
  'El Salvador',
  'Guatemala',
  'Honduras',
  'Mexico',
  'Nicaragua',
  'Panama',
  'United States'],
 'Ocea

Access spotify API and received playlist tracks information, but we found there are some playlists are not uploaded officially by Spotify. Here we choose to focus on official palylist, so we customize following function.

In [0]:
def collect_chart_ids(available_countries):
    chart_ids = []
    for continent in available_countries.keys():
        countries = available_countries[continent]
    
        for country in countries:
            playlist_name = country + ' Top 50'
            playlists = spotify.search(q = playlist_name, type = 'playlist')
            for playlist in playlists['playlists']['items']:
                
                # If not Spotify Official playlist
                if playlist['owner']['display_name'] != 'spotifycharts':
                    continue
                # If not Top 50 ex) Viral 50
                if playlist['name'] != playlist_name:
                    continue
                
                chart_ids.append((continent, country, playlist['id']))

    return chart_ids                

Get the playlist id

In [0]:
collect_chart_ids(get_available_countries())

[('Africa', 'South Africa', '37i9dQZEVXbMH2jvi6jvjk'),
 ('Asia', 'Hong Kong', '37i9dQZEVXbLwpL8TjsxOG'),
 ('Asia', 'India', '37i9dQZEVXbLZ52XmnySJg'),
 ('Asia', 'Indonesia', '37i9dQZEVXbObFQZ3JLcXt'),
 ('Asia', 'Israel', '37i9dQZEVXbJ6IpvItkve3'),
 ('Asia', 'Japan', '37i9dQZEVXbKXQ4mDTEBXq'),
 ('Asia', 'Malaysia', '37i9dQZEVXbJlfUljuZExa'),
 ('Asia', 'Philippines', '37i9dQZEVXbNBz9cRCSFkY'),
 ('Asia', 'Singapore', '37i9dQZEVXbK4gjvS1FjPY'),
 ('Asia', 'Taiwan', '37i9dQZEVXbMnZEatlMSiu'),
 ('Asia', 'Thailand', '37i9dQZEVXbMnz8KIWsvf9'),
 ('Asia', 'Vietnam', '37i9dQZEVXbLdGSmz6xilI'),
 ('Europe', 'Austria', '37i9dQZEVXbKNHh6NIXu36'),
 ('Europe', 'Belgium', '37i9dQZEVXbJNSeeHswcKB'),
 ('Europe', 'Bulgaria', '37i9dQZEVXbNfM2w2mq1B8'),
 ('Europe', 'Czech Republic', '37i9dQZEVXbIP3c3fqVrJY'),
 ('Europe', 'Denmark', '37i9dQZEVXbL3J0k32lWnN'),
 ('Europe', 'Estonia', '37i9dQZEVXbLesry2Qw2xS'),
 ('Europe', 'Finland', '37i9dQZEVXbMxcczTSoGwZ'),
 ('Europe', 'France', '37i9dQZEVXbIPWwFssbupI'),
 ('E

In [0]:
# check the number of playlist id we got
len(collect_chart_ids(get_available_countries()))

62

#### 2.2 Creat official country playlists

Load required libraries

In [0]:
import pandas as pd
from datetime import date, datetime

In [0]:
def collect_chart(continent, country, tracks):

    track_infos = []
    for idx, track in enumerate(tracks['items'], 1):
        track_info = {'continent': '', 'country' : '', 'rank': 0, 'song' : '', 'artist' : '', 'album' : '', 'song_id': '', 'artist_id': '', 'album_id': '', 'release_data': '', 'song_popularity' : 0, 'date': ''}

        track_info['rank'] = idx

        track_info['continent'] = continent
        track_info['country'] = country

        track_info['song'] = track['track']['name']
        track_info['song_id'] = track['track']['id']
        track_info['song_popularity'] = track['track']['popularity']

        track_info['album'] = track['track']['album']['name']
        track_info['album_id'] = track['track']['album']['id']
        
        try:
            track_info['release_date'] = datetime.strptime(track['track']['album']['release_date'], '%Y-%m-%d')
        except:
            track_info['release_date'] = datetime.strptime(track['track']['album']['release_date'], '%Y')

        track_info['artist'] = track['track']['artists'][0]['name']
        track_info['artist_id'] = track['track']['artists'][0]['id']

        track_info['date'] = date.today()
        track_infos.append(track_info)

    return track_infos

In [0]:
def playlist_track(self, playlist_id):
    """ returns a single artist given the artist's ID, URI or URL
        Parameters:
            - artist_id - an artist ID, URI or URL
    """
    return self._get('playlists/' + playlist_id + '/tracks')

In [0]:
import types
spotify.playlist_track = types.MethodType(playlist_track, spotify)

In [0]:
available_countries = get_available_countries()

#chart_ids = [(continent, country, playlist_id), ...]
chart_ids = collect_chart_ids(available_countries)

In [0]:
_columns = ['continent', 'country', 'rank', 'song', 'artist', 'album', 'release_date', 'song_popularity', 'song_id', 'artist_id', 'album_id', 'date']

global_chart = pd.DataFrame(columns = _columns)

for (continent, country, playlist_id) in chart_ids:
    
    tracks = spotify.playlist_track(playlist_id)
    dict_country_chart = collect_chart(continent, country, tracks)
    
    country_chart = pd.DataFrame(dict_country_chart, columns = _columns)
    
    global_chart = pd.concat([global_chart, country_chart], ignore_index = True)

retrying ...1secs


Show the dataset of United States Top 50

In [0]:
global_chart[global_chart['country'] == 'United States']

,continent,country,rank,song,artist,album,release_date,song_popularity,song_id,artist_id,album_id,date
2500,North America,United States,1,ROXANNE,Arizona Zervas,ROXANNE,2019-10-10,92,696DnlkuDOXcMAnKlTgXXK,0vRvGUQVUjytro0xpb26bs,6HJDrXs0hpebaRFKA1sF90,2019-11-26
2501,North America,United States,2,Circles,Post Malone,Hollywood's Bleeding,2019-09-06,99,21jGcNKet2qwijlDFuPiPb,246dkjvS1zLTtiykXe5h60,4g1ZRSobMefqF6nelkgibi,2019-11-26
2502,North America,United States,3,everything i wanted,Billie Eilish,everything i wanted,2019-11-13,95,3ZCTVFBt2Brf31RLEnCkWJ,6qqNVTkY8uBg9cP3Jd7DAH,4i3rAwPw7Ln2YrKDusaWyT,2019-11-26
2503,North America,United States,4,HIGHEST IN THE ROOM,Travis Scott,HIGHEST IN THE ROOM,2019-10-04,98,3eekarcy7kvN4yt5ZFzltW,0Y5tJX1MQlPlqiwlOH1tJY,2uDTi1PlpSpvAv7IRAoAEU,2019-11-26
2504,North America,United States,5,Bandit (with YoungBoy Never Broke Again),Juice WRLD,Bandit (with YoungBoy Never Broke Again),2019-10-04,94,6Gg1gjgKi2AK4e0qzsR7sd,4MCBfE4596Uoi2O4DtmEMz,3t6Z2qoBVCS4NHNI25XECH,2019-11-26
2505,North America,United States,6,Falling,Trevor Daniel,Falling,2018-10-05,87,4TnjEaWOeW0eKTKIEvJyCa,7uaIm6Pw7xplS8Dy06V6pT,1Czfd5tEby3DbdYNdqzrCa,2019-11-26
2506,North America,United States,7,BOP,DaBaby,KIRK,2019-09-27,89,6Ozh9Ok6h4Oi1wUSLtBseN,4r63FhuTkUYltbVAg5TQnk,1NsTSXjVNE7XmZ8PmyW0wl,2019-11-26
2507,North America,United States,8,Lose You To Love Me,Selena Gomez,Lose You To Love Me,2019-10-23,99,1HfMVBKM75vxSfsQ5VefZ5,0C8ZW7ezQVs4URX5aX7Kqx,3tBkjgxDqAwss76O1YHsSY,2019-11-26
2508,North America,United States,9,Watermelon Sugar,Harry Styles,Watermelon Sugar,2019-11-17,88,1e9oZCCiX42nJl0AcqriVo,6KImCVD70vtIoJWnq6nGn3,659e2eKbsMH0vYCs5qgFmy,2019-11-26
2509,North America,United States,10,Dance Monkey,Tones and I,Dance Monkey,2019-05-10,81,1rgnBhdG2JDFTbYkYRZAku,2NjfBq1NflQcKSeiDooVjY,31IDBea3eEs57a0joX6TjN,2019-11-26


In [0]:
global_chart.head(5)

,continent,country,rank,song,artist,album,release_date,song_popularity,song_id,artist_id,album_id,date
0,Africa,South Africa,1,Dance Monkey,Tones and I,Dance Monkey,2019-05-10,81,1rgnBhdG2JDFTbYkYRZAku,2NjfBq1NflQcKSeiDooVjY,31IDBea3eEs57a0joX6TjN,2019-11-26
1,Africa,South Africa,2,Memories,Maroon 5,Memories,2019-09-20,100,2b8fOow8UzyDFAE27YhOZM,04gDigrS5kc9YWfZHwBETP,3nR9B40hYLKLcR0Eph3Goc,2019-11-26
2,Africa,South Africa,3,Circles,Post Malone,Hollywood's Bleeding,2019-09-06,99,21jGcNKet2qwijlDFuPiPb,246dkjvS1zLTtiykXe5h60,4g1ZRSobMefqF6nelkgibi,2019-11-26
3,Africa,South Africa,4,ROXANNE,Arizona Zervas,ROXANNE,2019-10-10,92,696DnlkuDOXcMAnKlTgXXK,0vRvGUQVUjytro0xpb26bs,6HJDrXs0hpebaRFKA1sF90,2019-11-26
4,Africa,South Africa,5,everything i wanted,Billie Eilish,everything i wanted,2019-11-13,95,3ZCTVFBt2Brf31RLEnCkWJ,6qqNVTkY8uBg9cP3Jd7DAH,4i3rAwPw7Ln2YrKDusaWyT,2019-11-26


### 3. Add information of each song

#### 3.1 Get info of each track by song_id

In [0]:
def return_ids(requested_type, limits):
    """
    A generator that returns requested data(ids) in 50 pieces (Spotify API limits 50 ids per request)
    argument: requested_type = string('song_id' or 'artist_id')
    """
    ids = []
    for idx, song_id in enumerate((x for x in global_chart[requested_type].tolist()), 1):
        ids.append(song_id)
        if idx % limits == 0:
            yield ids
            ids.clear()

    if len(ids):
        yield ids

In [0]:
_columns = ['song_id', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'valence']
songs_df = pd.DataFrame(columns = _columns)

for x in return_ids('song_id', 50):
    
    songs_features = []
    song_infos = spotify.audio_features(x)
    
    for song_info in song_infos:
        song_features = {'song_id': '', 'acousticness': 0, 'danceability': 0, 'energy': 0, 'instrumentalness': 0, 'liveness': 0 , 'loudness': 0, 'speechiness': 0, 'valence': 0}
        song_features['song_id'] = song_info['id']
        song_features['acousticness'] = song_info['acousticness']
        song_features['danceability'] = song_info['danceability']
        song_features['energy'] = song_info['energy']
        song_features['instrumentalness'] = song_info['instrumentalness']
        song_features['liveness'] = song_info['liveness']
        song_features['loudness'] = song_info['loudness']
        song_features['speechiness'] = song_info['speechiness']
        song_features['valence'] = song_info['valence']
        
        songs_features.append(song_features)
    
    temp_songs_df = pd.DataFrame(songs_features, columns = _columns)
    songs_df = pd.concat([songs_df, temp_songs_df], ignore_index = True)
songs_df.head(10)

,song_id,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence
0,1rgnBhdG2JDFTbYkYRZAku,0.6880,0.825,0.593,0.000161,0.1700,-6.401,0.0988,0.5400
1,2b8fOow8UzyDFAE27YhOZM,0.8370,0.764,0.320,0.000000,0.0822,-7.209,0.0546,0.5750
2,21jGcNKet2qwijlDFuPiPb,0.1920,0.695,0.762,0.002440,0.0863,-3.497,0.0395,0.5530
3,696DnlkuDOXcMAnKlTgXXK,0.0522,0.621,0.601,0.000000,0.4600,-5.616,0.1480,0.4570
4,3ZCTVFBt2Brf31RLEnCkWJ,0.9020,0.704,0.225,0.657000,0.1060,-14.454,0.0994,0.2430
5,1HfMVBKM75vxSfsQ5VefZ5,0.5760,0.505,0.340,0.000000,0.2100,-9.005,0.0438,0.0916
6,70eFcWOvlMObDhURTqT4Fv,0.1240,0.640,0.648,0.000000,0.0802,-8.113,0.1870,0.5480
7,3eekarcy7kvN4yt5ZFzltW,0.0546,0.598,0.427,0.000006,0.2100,-8.764,0.0317,0.0605
8,6WrI0LAC5M1Rw2MnX2ZvEg,0.0125,0.794,0.793,0.000000,0.0952,-4.521,0.0842,0.6770
9,7qEHsqek33rTcFNT9PFqLf,0.7510,0.501,0.405,0.000000,0.1050,-5.679,0.0319,0.4460


#### 3.2 Insert track info to our playlist dataset

In [0]:
global_chart = pd.concat([global_chart, songs_df], axis = 1)
global_chart = global_chart.loc[:,~global_chart.columns.duplicated()]

In [0]:
global_chart.tail(10)

,continent,country,rank,song,artist,album,release_date,song_popularity,song_id,artist_id,album_id,date,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence
3090,Oceania,New Zealand,41,Liar,Camila Cabello,Liar,2019-09-04,94,7LzouaWGFCy4tkXDOOnEyM,4nDoRrQiYLoBzwC5BhVJzF,3xgS4rfthlsvhW7J2WLiiR,2019-11-26,0.016900,0.740,0.498,0.002820,0.3190,-6.684,0.0456,0.652
3091,Oceania,New Zealand,42,Mean It,Lauv,~how i'm feeling~,2019-11-14,87,3GRSqlALWISqLeNncZMbpX,5JZ7CnR6gTvEMKX4g70Amv,4aoWQU2gSUjl4AZeCPSo1b,2019-11-26,0.040700,0.746,0.450,0.000000,0.1720,-8.543,0.0872,0.336
3092,Oceania,New Zealand,43,Sundown,SIX60,Six60,2019-11-08,57,4WEAdAPGiyI6LIMlHGfhdZ,1caoBfXJrbKCwIaTzGkyHn,7MenxXNX8NaBCcQtu40ccR,2019-11-26,0.402000,0.794,0.553,0.001600,0.2060,-7.304,0.0434,0.485
3093,Oceania,New Zealand,44,Follow God,Kanye West,JESUS IS KING,2019-10-25,93,2QpGZOhTCHHiKmpSO9FW4h,5K4W6rqBFWDnAN6FQUkS6x,0FgZKfoU2Br5sHOfvZKTI9,2019-11-26,0.000248,0.529,0.900,0.000012,0.0426,-3.503,0.4180,0.434
3094,Oceania,New Zealand,45,No Guidance (feat. Drake),Chris Brown,Indigo,2019-06-28,88,6XHVuErjQ4XNm6nDPVCxVX,7bXgB6jMjp9ATFy66eO08Z,1BfLzaTFI5qKsAAk0Ae6aV,2019-11-26,0.123000,0.698,0.454,0.000000,0.1550,-6.756,0.1510,0.137
3095,Oceania,New Zealand,46,It's You,Ali Gatie,YOU,2019-11-08,86,5DqdesEfbRyOlSS3Tf6c29,4rTv3Ejc7hKMtmoBOK1B4T,63Kr9mdWxT7mB2AKgf4rCa,2019-11-26,0.374000,0.732,0.463,0.000000,0.1940,-6.972,0.0287,0.397
3096,Oceania,New Zealand,47,Bad N Bouj,Hp Boyz,Bad N Bouj,2019-09-16,68,3XV2EL4JquJ7BEA4Tyi8Gl,0JlQaGJZfrqYeZG1AQICxp,4uzDnzu50ShEIxahvu0R3o,2019-11-26,0.470000,0.938,0.567,0.000000,0.0951,-9.180,0.2780,0.702
3097,Oceania,New Zealand,48,Eastside (with Halsey & Khalid),benny blanco,FRIENDS KEEP SECRETS,2018-12-07,80,7FGq80cy8juXBCD2nrqdWU,5CiGnKThu5ctn9pBxv7DGa,7dQ734EW0iLvQfF6vBFNiZ,2019-11-26,0.549000,0.632,0.686,0.000000,0.2000,-7.665,0.2430,0.329
3098,Oceania,New Zealand,49,Misunderstood,Youngn Lipz,Misunderstood,2019-10-19,69,0aZMdAA6pg0IdH1r9MF5sT,36PxHmbx5QrRTpuCaw4FuV,6crzqCZhOc8KON5m6zBNlO,2019-11-26,0.114000,0.882,0.674,0.000000,0.3480,-5.251,0.1080,0.436
3099,Oceania,New Zealand,50,Talk (feat. Disclosure),Khalid,Free Spirit,2019-04-05,84,6TqXcAFInzjp0bODyvrWEq,6LuN9FCkKOj5PcnpouEgny,6KT8x5oqZJl9CcnM66hddo,2019-11-26,0.051000,0.901,0.400,0.000000,0.0599,-8.575,0.1260,0.346


### 4. Add information of each artist

#### 4.1 Get info of each artist by artist_id

In [0]:
_columns = ['artist_id', 'genre', 'artist_popularity', 'followers']
artists_df = pd.DataFrame(columns = _columns)

for x in return_ids('artist_id', 50):
    
    artists_features = []
    
    artist_infos = spotify.artists(x)

    for artist_info in artist_infos['artists']:
        artist_features = {'artist_id': '', 'genre': '', 'artist_popularity': 0, 'followers': 0}
        artist_features['artist_id'] = artist_info['id']
        try:
            artist_features['genre'] = artist_info['genres'][0]
            #artist_features['genre'] = artist_info['genres'][0].split(' ')[0]
        except:
            # TTD : Searching Genre from google? 
            artist_features['genre'] = None
            #print(artist_info['genres'], artist_info['name'])
        artist_features['artist_popularity'] = artist_info['popularity']
        artist_features['followers'] = artist_info['followers']['total']
        
        artists_features.append(artist_features)
    
    temp_artists_df = pd.DataFrame(artists_features, columns = _columns)
    artists_df = pd.concat([artists_df, temp_artists_df], ignore_index = True)

artists_df.head(100)

,artist_id,genre,artist_popularity,followers
0,2NjfBq1NflQcKSeiDooVjY,australian pop,90,258076
1,04gDigrS5kc9YWfZHwBETP,pop,93,20913384
2,246dkjvS1zLTtiykXe5h60,dfw rap,100,19579289
3,0vRvGUQVUjytro0xpb26bs,pop rap,85,103711
4,6qqNVTkY8uBg9cP3Jd7DAH,electropop,96,16112471
...,...,...,...,...
95,3HqSLMAZ3g3d5poNaI7GOU,k-pop,75,1009205
96,2SrSdSvpminqmStGELCSNd,grime,81,1108322
97,73Np75Wv2tju61Eo9Zw4IR,broadway,76,215873
98,69GGBxA162lTqCwzJG5jLp,electropop,90,14537838


In [0]:
artists_df.shape

(3100, 4)

#### 4.2 Insert artist info to our playlist dataset

In [0]:
global_chart = pd.concat([global_chart, artists_df], axis = 1)
global_chart = global_chart.loc[:,~global_chart.columns.duplicated()]

In [0]:
global_chart.head(10)

,continent,country,rank,song,artist,album,release_date,song_popularity,song_id,artist_id,album_id,date,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,genre,artist_popularity,followers
0,Africa,South Africa,1,Dance Monkey,Tones and I,Dance Monkey,2019-05-10,81,1rgnBhdG2JDFTbYkYRZAku,2NjfBq1NflQcKSeiDooVjY,31IDBea3eEs57a0joX6TjN,2019-11-26,0.6880,0.825,0.593,0.000161,0.1700,-6.401,0.0988,0.5400,australian pop,90,258076
1,Africa,South Africa,2,Memories,Maroon 5,Memories,2019-09-20,100,2b8fOow8UzyDFAE27YhOZM,04gDigrS5kc9YWfZHwBETP,3nR9B40hYLKLcR0Eph3Goc,2019-11-26,0.8370,0.764,0.320,0.000000,0.0822,-7.209,0.0546,0.5750,pop,93,20913384
2,Africa,South Africa,3,Circles,Post Malone,Hollywood's Bleeding,2019-09-06,99,21jGcNKet2qwijlDFuPiPb,246dkjvS1zLTtiykXe5h60,4g1ZRSobMefqF6nelkgibi,2019-11-26,0.1920,0.695,0.762,0.002440,0.0863,-3.497,0.0395,0.5530,dfw rap,100,19579289
3,Africa,South Africa,4,ROXANNE,Arizona Zervas,ROXANNE,2019-10-10,92,696DnlkuDOXcMAnKlTgXXK,0vRvGUQVUjytro0xpb26bs,6HJDrXs0hpebaRFKA1sF90,2019-11-26,0.0522,0.621,0.601,0.000000,0.4600,-5.616,0.1480,0.4570,pop rap,85,103711
4,Africa,South Africa,5,everything i wanted,Billie Eilish,everything i wanted,2019-11-13,95,3ZCTVFBt2Brf31RLEnCkWJ,6qqNVTkY8uBg9cP3Jd7DAH,4i3rAwPw7Ln2YrKDusaWyT,2019-11-26,0.9020,0.704,0.225,0.657000,0.1060,-14.454,0.0994,0.2430,electropop,96,16112471
5,Africa,South Africa,6,Lose You To Love Me,Selena Gomez,Lose You To Love Me,2019-10-23,99,1HfMVBKM75vxSfsQ5VefZ5,0C8ZW7ezQVs4URX5aX7Kqx,3tBkjgxDqAwss76O1YHsSY,2019-11-26,0.5760,0.505,0.340,0.000000,0.2100,-9.005,0.0438,0.0916,dance pop,92,17901101
6,Africa,South Africa,7,Beautiful People (feat. Khalid),Ed Sheeran,No.6 Collaborations Project,2019-07-12,87,70eFcWOvlMObDhURTqT4Fv,6eUKZXaKkcviH0Ku9w2n3V,3oIFxDIo2fwuk4lwCmFZCx,2019-11-26,0.1240,0.640,0.648,0.000000,0.0802,-8.113,0.1870,0.5480,pop,97,54510835
7,Africa,South Africa,8,HIGHEST IN THE ROOM,Travis Scott,HIGHEST IN THE ROOM,2019-10-04,98,3eekarcy7kvN4yt5ZFzltW,0Y5tJX1MQlPlqiwlOH1tJY,2uDTi1PlpSpvAv7IRAoAEU,2019-11-26,0.0546,0.598,0.427,0.000006,0.2100,-8.764,0.0317,0.0605,pop rap,97,7129745
8,Africa,South Africa,9,Don't Start Now,Dua Lipa,Don't Start Now,2019-10-31,96,6WrI0LAC5M1Rw2MnX2ZvEg,6M2wZ9GZgrQXHCFfjv46we,0ix3XtPV1LwmZADsprKxcp,2019-11-26,0.0125,0.794,0.793,0.000000,0.0952,-4.521,0.0842,0.6770,dance pop,90,14229251
9,Africa,South Africa,10,Someone You Loved,Lewis Capaldi,Divinely Uninspired To A Hellish Extent,2019-05-17,96,7qEHsqek33rTcFNT9PFqLf,4GNC7GD6oZMSxPGyXy4MNB,5658aM19fA3JVwTK6eQX70,2019-11-26,0.7510,0.501,0.405,0.000000,0.1050,-5.679,0.0319,0.4460,pop,90,1624700


### 5. Add information of each album

#### 5.1 Get info of each album by album_id

In [0]:
_columns = ['album_id', 'album_popularity']
albums_df = pd.DataFrame(columns = _columns)

for x in return_ids('album_id', 20):
    
    albums_features = []
    albums_infos = spotify.albums(x)

    for album_info in albums_infos['albums']:
        album_features = {'album_id': '', 'album_popularity': 0}
        album_features['album_id'] = album_info['id']
        album_features['album_popularity'] = album_info['popularity']
        
        albums_features.append(album_features)
    
    temp_albums_df = pd.DataFrame(albums_features, columns = _columns)
    albums_df = pd.concat([albums_df, temp_albums_df], ignore_index = True)

retrying ...1secs


In [0]:
albums_df

,album_id,album_popularity
0,31IDBea3eEs57a0joX6TjN,72
1,3nR9B40hYLKLcR0Eph3Goc,90
2,4g1ZRSobMefqF6nelkgibi,100
3,6HJDrXs0hpebaRFKA1sF90,82
4,4i3rAwPw7Ln2YrKDusaWyT,86
...,...,...
3095,63Kr9mdWxT7mB2AKgf4rCa,78
3096,4uzDnzu50ShEIxahvu0R3o,58
3097,7dQ734EW0iLvQfF6vBFNiZ,75
3098,6crzqCZhOc8KON5m6zBNlO,59


In [0]:
global_chart.tail(50)

,continent,country,rank,song,artist,album,release_date,song_popularity,song_id,artist_id,album_id,date,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,genre,artist_popularity,followers
3050,Oceania,New Zealand,1,ROXANNE,Arizona Zervas,ROXANNE,2019-10-10,92,696DnlkuDOXcMAnKlTgXXK,0vRvGUQVUjytro0xpb26bs,6HJDrXs0hpebaRFKA1sF90,2019-11-26,0.052200,0.621,0.601,0.000000,0.4600,-5.616,0.1480,0.4570,pop,85,103711
3051,Oceania,New Zealand,2,Dance Monkey,Tones and I,Dance Monkey,2019-05-10,81,1rgnBhdG2JDFTbYkYRZAku,2NjfBq1NflQcKSeiDooVjY,31IDBea3eEs57a0joX6TjN,2019-11-26,0.688000,0.825,0.593,0.000161,0.1700,-6.401,0.0988,0.5400,australian,90,258076
3052,Oceania,New Zealand,3,Circles,Post Malone,Hollywood's Bleeding,2019-09-06,99,21jGcNKet2qwijlDFuPiPb,246dkjvS1zLTtiykXe5h60,4g1ZRSobMefqF6nelkgibi,2019-11-26,0.192000,0.695,0.762,0.002440,0.0863,-3.497,0.0395,0.5530,dfw,100,19579289
3053,Oceania,New Zealand,4,Memories,Maroon 5,Memories,2019-09-20,100,2b8fOow8UzyDFAE27YhOZM,04gDigrS5kc9YWfZHwBETP,3nR9B40hYLKLcR0Eph3Goc,2019-11-26,0.837000,0.764,0.320,0.000000,0.0822,-7.209,0.0546,0.5750,pop,93,20913384
3054,Oceania,New Zealand,5,everything i wanted,Billie Eilish,everything i wanted,2019-11-13,95,3ZCTVFBt2Brf31RLEnCkWJ,6qqNVTkY8uBg9cP3Jd7DAH,4i3rAwPw7Ln2YrKDusaWyT,2019-11-26,0.902000,0.704,0.225,0.657000,0.1060,-14.454,0.0994,0.2430,electropop,96,16112471
3055,Oceania,New Zealand,6,Please Don't Go,SIX60,Six60,2019-11-08,60,5caY6lIFeI2du0TpgCSRP1,1caoBfXJrbKCwIaTzGkyHn,7MenxXNX8NaBCcQtu40ccR,2019-11-26,0.364000,0.700,0.520,0.000000,0.2010,-6.427,0.1920,0.7040,nz,73,267634
3056,Oceania,New Zealand,7,hot girl bummer,blackbear,hot girl bummer,2019-08-23,97,7aiClxsDWFRQ0Kzk5KI5ku,2cFrymmkijnjDg9SS92EPM,37CqAwxTungNxKpIK5vSgE,2019-11-26,0.128000,0.778,0.559,0.000000,0.3990,-7.109,0.0776,0.6780,pop,89,2540610
3057,Oceania,New Zealand,8,Catching Feelings (feat. SIX60),Drax Project,Drax Project,2019-09-27,68,0gCNs3jCCDhObAK0MRghtv,6S0IvKlvPMX1RtAYtVpUV8,5gdsguQxwQQlpTi1SU3tbY,2019-11-26,0.607000,0.397,0.524,0.000000,0.1570,-5.846,0.1030,0.3430,australian,70,43769
3058,Oceania,New Zealand,9,Don't Start Now,Dua Lipa,Don't Start Now,2019-10-31,96,6WrI0LAC5M1Rw2MnX2ZvEg,6M2wZ9GZgrQXHCFfjv46we,0ix3XtPV1LwmZADsprKxcp,2019-11-26,0.012500,0.794,0.793,0.000000,0.0952,-4.521,0.0842,0.6770,dance,90,14229251
3059,Oceania,New Zealand,10,The Greatest,SIX60,Six60,2019-11-08,58,2OmbLtdHJn3u9haCQXODl6,1caoBfXJrbKCwIaTzGkyHn,7MenxXNX8NaBCcQtu40ccR,2019-11-26,0.330000,0.623,0.628,0.000019,0.1950,-5.839,0.0479,0.3380,nz,73,267634


#### 5.2 Insert album info to our playlist dataset

In [0]:
global_chart = pd.concat([global_chart, albums_df], axis = 1)
global_chart = global_chart.loc[:,~global_chart.columns.duplicated()]

In [0]:
global_chart[global_chart['country'] == 'United States']

,continent,country,rank,song,artist,album,release_date,song_popularity,song_id,artist_id,album_id,date,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,genre,artist_popularity,followers,album_popularity
2500,North America,United States,1,ROXANNE,Arizona Zervas,ROXANNE,2019-10-10,92,696DnlkuDOXcMAnKlTgXXK,0vRvGUQVUjytro0xpb26bs,6HJDrXs0hpebaRFKA1sF90,2019-11-26,0.052200,0.621,0.601,0.000000,0.4600,-5.616,0.1480,0.4570,pop,85,103711,82
2501,North America,United States,2,Circles,Post Malone,Hollywood's Bleeding,2019-09-06,99,21jGcNKet2qwijlDFuPiPb,246dkjvS1zLTtiykXe5h60,4g1ZRSobMefqF6nelkgibi,2019-11-26,0.192000,0.695,0.762,0.002440,0.0863,-3.497,0.0395,0.5530,dfw,100,19579289,100
2502,North America,United States,3,everything i wanted,Billie Eilish,everything i wanted,2019-11-13,95,3ZCTVFBt2Brf31RLEnCkWJ,6qqNVTkY8uBg9cP3Jd7DAH,4i3rAwPw7Ln2YrKDusaWyT,2019-11-26,0.902000,0.704,0.225,0.657000,0.1060,-14.454,0.0994,0.2430,electropop,96,16112471,86
2503,North America,United States,4,HIGHEST IN THE ROOM,Travis Scott,HIGHEST IN THE ROOM,2019-10-04,98,3eekarcy7kvN4yt5ZFzltW,0Y5tJX1MQlPlqiwlOH1tJY,2uDTi1PlpSpvAv7IRAoAEU,2019-11-26,0.054600,0.598,0.427,0.000006,0.2100,-8.764,0.0317,0.0605,pop,97,7129745,89
2504,North America,United States,5,Bandit (with YoungBoy Never Broke Again),Juice WRLD,Bandit (with YoungBoy Never Broke Again),2019-10-04,94,6Gg1gjgKi2AK4e0qzsR7sd,4MCBfE4596Uoi2O4DtmEMz,3t6Z2qoBVCS4NHNI25XECH,2019-11-26,0.068700,0.474,0.631,0.000000,0.1320,-5.884,0.3430,0.4250,chicago,93,6151768,84
2505,North America,United States,6,Falling,Trevor Daniel,Falling,2018-10-05,87,4TnjEaWOeW0eKTKIEvJyCa,7uaIm6Pw7xplS8Dy06V6pT,1Czfd5tEby3DbdYNdqzrCa,2019-11-26,0.123000,0.784,0.430,0.000000,0.0887,-8.756,0.0364,0.2360,alternative,76,62372,78
2506,North America,United States,7,BOP,DaBaby,KIRK,2019-09-27,89,6Ozh9Ok6h4Oi1wUSLtBseN,4r63FhuTkUYltbVAg5TQnk,1NsTSXjVNE7XmZ8PmyW0wl,2019-11-26,0.189000,0.769,0.787,0.000000,0.1290,-3.909,0.3670,0.8360,nc,93,1314453,89
2507,North America,United States,8,Lose You To Love Me,Selena Gomez,Lose You To Love Me,2019-10-23,99,1HfMVBKM75vxSfsQ5VefZ5,0C8ZW7ezQVs4URX5aX7Kqx,3tBkjgxDqAwss76O1YHsSY,2019-11-26,0.576000,0.505,0.340,0.000000,0.2100,-9.005,0.0438,0.0916,dance,92,17901101,89
2508,North America,United States,9,Watermelon Sugar,Harry Styles,Watermelon Sugar,2019-11-17,88,1e9oZCCiX42nJl0AcqriVo,6KImCVD70vtIoJWnq6nGn3,659e2eKbsMH0vYCs5qgFmy,2019-11-26,0.122000,0.548,0.816,0.000000,0.3350,-4.209,0.0465,0.5570,dance,87,5347831,80
2509,North America,United States,10,Dance Monkey,Tones and I,Dance Monkey,2019-05-10,81,1rgnBhdG2JDFTbYkYRZAku,2NjfBq1NflQcKSeiDooVjY,31IDBea3eEs57a0joX6TjN,2019-11-26,0.688000,0.825,0.593,0.000161,0.1700,-6.401,0.0988,0.5400,australian,90,258076,72
